# Step 1

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import ast
stop_words = set(stopwords.words('english'))
from nltk.tokenize import RegexpTokenizer
seperator = '#;'

In [2]:
bingLui = [bingline.rstrip("\n") for bingline in open("../../Bing Liu/Hu and Liu Sentiment Lexicon/positive-words.txt", encoding='utf8')]
bingLuiArray = []
for row in bingLui:
    bingLuiArray.append(row)

In [3]:
lines = [line.rstrip("\n") for line in open("../TrainingData/trainingData.txt", encoding='utf8')]

In [4]:
def wordtokanize(sentence):
    word_tokens = word_tokenize(sentence)
    word_tokens = [token.lower() for token in word_tokens]
    return word_tokens

In [5]:
def countOccurance(sequence_of_sentences):
    counts = Counter()
    for word in sequence_of_sentences:
        counts[word] += 1
    return dict(counts)     

In [6]:
def adjusent_Bygram (array,token):
    globalArray =[]    
    length = 0    
    stringlength = len(array)
    tokenArray = []
    for i in array:    
        if(i == token):
            if( length ==0):                
                info = {array[length],array[length+1]}
                tokenArray.append(info)
            if( length == stringlength-1):                 
                info = {array[length-1],array[length]}
                tokenArray.append(info)
            if ((length !=0) & (length != stringlength-1)):
                info = {array[length],array[length-1]}
                info1 = {array[length],array[length+1]}
                tokenArray.append(info)
                tokenArray.append(info1)                
        length = length+1;   
    return tokenArray

In [7]:
def generateAdjuacentByGram(wordToken,token):
    globalArray =[]
    tokenLength = len(token)
    if(tokenLength>0):
        for i in token:            
            a1 = adjusent_Bygram (wordToken,i)
            globalArray.append(a1)    
    return globalArray

In [8]:
f = open("../TrainigDumpFile/wordtokanize1.txt","w", encoding='utf8') 
tempPositiveData = []
for row in lines:
    liuPositiveLexiconWords =[]
    a = np.array(row.split("\t"))  
    trainingLable = a[0]
    sentence = a[1]    
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(sentence)
    a3 = [w for w in word_tokens if not w in stop_words]
    tempLiuPositiveLexiconWords =[]
    for v,k in nltk.pos_tag(a3):
        for indBing in bingLuiArray:
            if(v == indBing):
                tempLiuPositiveLexiconWords.append(v)                
                break;
    occouredPositiveLexicon = list(set(tempLiuPositiveLexiconWords))    
    posTagForPositiveOccouredLexicon = nltk.pos_tag(occouredPositiveLexicon)
    bygram = generateAdjuacentByGram(a3,occouredPositiveLexicon)
    
    posLexicon = countOccurance(tempLiuPositiveLexiconWords)   
    if(len(posLexicon) > 0):
        liuPositiveLexiconWords.append(posLexicon)
    tempPositiveData.append(countOccurance(tempLiuPositiveLexiconWords))
    f.writelines(str(sentence))
    f.writelines(str(seperator))
    f.writelines(str(liuPositiveLexiconWords))
    f.writelines(str(seperator))
    f.writelines(str(bygram))
    f.writelines(str(seperator))
    f.writelines(str(posTagForPositiveOccouredLexicon))
    f.writelines(str(seperator))
    f.writelines(str(trainingLable))
    f.writelines('\n')
f.close() 

In [9]:
from collections import defaultdict
d = defaultdict(int)
for i in tempPositiveData:
    for k,v in i.items():
        d[k] += v

In [10]:
tempTokanizedData = [tempTokanizedData.rstrip("\n") for tempTokanizedData in open("../TrainigDumpFile/wordtokanize1.txt", encoding='utf8')]

In [11]:
for indrow in tempTokanizedData:
    sentence = indrow.split(';')[0]
    positiveWordCount = indrow.split(';')[1]

In [12]:
bingLiuNegativeWord = [indbingline.rstrip("\n") for indbingline in open("../../Bing Liu/Hu and Liu Sentiment Lexicon/negative-words.txt")]
bingLiuNegativeArray = []
for row in bingLiuNegativeWord:
    bingLiuNegativeArray.append(row)

In [13]:
f = open("../TrainigDumpFile/wordtokanize.txt","w", encoding='utf8') 
for indrow in tempTokanizedData:
    liuNegativeLexiconWords =[]
    sentence = indrow.split(seperator)[0]
    positiveWordCount = indrow.split(seperator)[1]
    positiveBigram = indrow.split(seperator)[2]
    positivePosTag = indrow.split(seperator)[3]
    trainingLable = indrow.split(seperator)[4]
    tempLiuNegativeLexiconWords =[]   
    sentence = sentence.lower()
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(sentence)
    a3 = [w for w in word_tokens if not w in stop_words]   
    for v,k in nltk.pos_tag(a3):
        for indBing in bingLiuNegativeArray:
            if(v == indBing):
                tempLiuNegativeLexiconWords.append(v)
                break;
    occouredNegativeLexicon = list(set(tempLiuNegativeLexiconWords))    
    negativePosTag = nltk.pos_tag(occouredNegativeLexicon)
    
    negLexicon = countOccurance(tempLiuNegativeLexiconWords)
    
    if(len(negLexicon) > 0):
        liuNegativeLexiconWords.append(negLexicon)
    negativeBigram = generateAdjuacentByGram(a3,occouredNegativeLexicon)
    positiveWordArray = []
    
    f.writelines(sentence)
    f.writelines(str(seperator))
    f.writelines(positiveWordCount)
    f.writelines(str(seperator))
    f.writelines(str(liuNegativeLexiconWords))
    f.writelines(str(seperator))
    f.writelines(str(positiveBigram))
    f.writelines(str(seperator))
    f.writelines(str(negativeBigram))
    f.writelines(str(seperator))
    f.writelines(str(positivePosTag))
    f.writelines(str(seperator))
    f.writelines(str(negativePosTag))
    f.writelines(str(seperator))
    f.writelines(str(trainingLable))
    f.writelines('\n')
f.close() 

In [14]:
fileData = [fileData.rstrip("\n") for fileData in open("../TrainigDumpFile/wordtokanize.txt", encoding='utf8')]

In [15]:
f = open("../TrainigDumpFile/score1.txt","w",encoding='utf8') 
for row1 in fileData:
    
    sentence = row1.split(seperator)[0]  
    positiveLexiconWordCount = row1.split(seperator)[1] 
    negtiveLexiconWordCount = row1.split(seperator)[2] 
    temppositiveByGram = row1.split(seperator)[3]
    tempnegativeByGram = row1.split(seperator)[4]
    trainingLable = row1.split(seperator)[7]
    
    _positiveByGram = str(temppositiveByGram)[1:-1]
    _negativeByGram = str(tempnegativeByGram)[1:-1]    
    positiveByGram = np.array(_positiveByGram)
    
    
    count  = 0;
    finalCount =0
    if(positiveByGram != ''):
        positiveByGram = ast.literal_eval(str(positiveByGram))
        for i in positiveByGram:
            count  = count +1  
        finalCount =0.8
        finalCount = finalCount +count
        ##print(finalCount)      
    f.writelines(sentence)
    f.writelines(str(seperator))  
    f.writelines(temppositiveByGram)
    f.writelines(str(seperator)) 
    f.writelines(tempnegativeByGram)
    f.writelines(str(seperator)) 
    f.writelines(str(finalCount))
    f.writelines(str(seperator)) 
    f.writelines(trainingLable)
    f.writelines(str(seperator)) 
    f.writelines(str(positiveLexiconWordCount))
    f.writelines(str(seperator)) 
    f.writelines(str(negtiveLexiconWordCount))
    f.writelines('\n')
f.close() 

In [16]:
scoreData = [scoreData.rstrip("\n") for scoreData in open("../TrainigDumpFile/score1.txt",encoding='utf8')]

In [17]:
f = open("../TrainigDumpFile/score.txt","w",encoding='utf8') 
for row1 in scoreData:    
    sentence = row1.split(seperator)[0]  
    _positiveByGram = row1.split(seperator)[1]  
    _negativeByGram1 = row1.split(seperator)[2]  
    positiveScore = row1.split(seperator)[3] 
    trainingLable = row1.split(seperator)[4] 
    _negativeByGram = str(_negativeByGram1)[1:-1]    
    negativeByGram = np.array(_negativeByGram) 
    positiveLexiconWordCount = row1.split(seperator)[5] 
    negtiveLexiconWordCount = row1.split(seperator)[6] 
     
    count  = 0;
    finalCount =0
    if(negativeByGram != ''):
        negativeByGram = ast.literal_eval(str(negativeByGram))
        for i in negativeByGram:
            count  = count +1  
        finalCount =0.8
        finalCount = finalCount +count
        ##print(finalCount)      
    f.writelines(sentence)
    f.writelines(str(seperator)) 
    f.writelines(positiveScore)
    f.writelines(str(seperator)) 
    f.writelines(str(finalCount))
    f.writelines(str(seperator)) 
    f.writelines(_positiveByGram)
    f.writelines(str(seperator)) 
    f.writelines(_negativeByGram1)
    f.writelines(str(seperator)) 
    f.writelines(trainingLable)    
    f.writelines(str(seperator)) 
    f.writelines(positiveLexiconWordCount)
    f.writelines(str(seperator)) 
    f.writelines(negtiveLexiconWordCount)    
    f.writelines('\n')
f.close() 

In [18]:
print("---------------END-------------------")

---------------END-------------------
